##### Grading Feedback

# Question 0 (-2 If not answered)
Please provide the following the data so we can verify your github information and ensure accurate grading:
- Your Name:  Yunhan Zhang
- Your SU ID: 405379315

# IST 718: Big Data Analytics

- Professors: 
  - Willard Williamson <wewillia@syr.edu>
  - Emory Creel <emcreel@g.syr.edu>
- Faculty Assistants: 
  - Warren Justin Fernandes <wjfernan@syr.edu>
  - Ruchita Hiteshkumar Harsora <	rharsora@g.syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers from your classmates.  Short code snippets are allowed from the internet.  Code from the class text books or class provided code can be copied in its entirety.__
- Google Colab is the official class runtime environment so you should test your code on Colab before submission.
- Do not modify cells marked as grading cells or marked as do not modify.
- Before submitting your work, remember to check for run time errors with the following procedure:
`Runtime `$\rightarrow$ Factory reset runtime followed by Runtime $\rightarrow$ Run All.  All runtime errors will result in a minimum penalty of half off.
- All plots shall include descriptive title and axis labels.  Plot legends shall be included where possible.  Unless stated otherwise, plots can be made using any Python plotting package.
- Grading feedback cells are there for graders to provide feedback to students.  Don't change or remove grading feedback cells.
- Don't add or remove files from your git repo.
- Do not change file names in your repo.  This also means don't change the title of the ipython notebook.
- You are free to add additional code cells around the cells marked `your code here`.
- import * is not allowed because it is considered a very bad coding practice and in some cases can result in a significant delay (which slows down the grading process) in loading imports.  For example, the statement `from sympy import *` is not allowed.  You must import the specific packages that you need. 
- The graders reserve the right to deduct points for subjective things we see with your code.  For example, if we ask you to create a pandas data frame to display values from an investigation and you hard code the values, we will take points off for that.  This is only one of many different things we could find in reviewing your code.  In general, write your code like you are submitting it for a code peer review in industry.  
- Level of effort is part of our subjective grading.  For example, in cases where we ask for a more open ended investigation, some students put in significant effort and some students do the minimum possible to meet requirements.  In these cases, we may take points off for students who did not put in much effort as compared to students who put in a lot of effort.  We feel that the students who did a better job deserve a better grade.  We reserve the right to invoke level of effort grading at any time.
- Your notebook must run from start to finish without requiring manual input by the graders.  For example, do not mount your personal Google drive in your notebook as this will require graders to perform manual steps.  In short, your notebook should run from start to finish with no runtime errors and no need for graders to perform any manual steps.

I was very disappointed with the linear regression model accuracy releted to the insurance data set in homework 3.  In this homework, we will revisit the insurance data set and try to improve prediction scores.  Specifically, we will use random forest, gradient boosting trees, and deep learning to see if we can improve upon the scores achieved in homework 3.  Part 1 of the assignment will explore random forest and GBT.  Part 2 of the assignment will use deep learning.

In [1]:
# Grading Cell
enable_grid_search = False

The following cell is used to read the insurance data set into the colab environment.  Do not change or modify the following cell.

In [2]:
%%bash
# Do not change or modify this cell
# Need to install pyspark
# if pyspark is already installed, will print a message indicating pyspark already installed
pip install pyspark &> /dev/null

# Download the data files from github
# If the data file does not exist in the colab environment
data_file_1=insurance.csv

if [[ ! -f ./${data_file_1} ]]; then 
   # download the data file from github and save it in this colab environment instance
   wget https://raw.githubusercontent.com/wewilli1/ist718_data/master/${data_file_1} &> /dev/null
fi

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import feature
from pyspark.ml import classification
from pyspark.sql import functions as fn
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, \
    MulticlassClassificationEvaluator, \
    RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.linalg import DenseVector
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import subprocess
from pyspark.sql import SparkSession
import glob

spark = SparkSession.builder.master("local").appName("MLP Demo").config("spark.master", "local[*]").getOrCreate()
sc = spark.sparkContext
spark

Your grade for grid search problems in this assignment will be determined in part on level of effort and your model performance results as compared to other students in the class.

# Question 0 (0 pts)
Copy the hard coded MSE scores from part 1 question 9 below (replace the code below from part 1 question 9).

In [4]:
# uncomment and hard code the following variables using output from above.  
# You can copy this code for use in part 2
hc_rf_train_mse = 17613516.100674648
hc_rf_validation_mse = 25511794.8633418
hc_gbt_train_mse = 24838572.975433294
hc_gbt_validation_mse = 28457932.60573362

# Question 1 (0 pts)
- This question is worth 0 points because you can just copy your code from part 1 question 1.  
- Read the insurance data file into a spark data frame named `medical_df`.  Drop any rows that contain NAN / Null values.  Check the schema and fix if needed.  Perform needed feature engineering using **only** a string indexer to get ready for training decision trees.  One hot encoding is not needed for random forest - do not use one hot encoding or any other transformations other than string indexing. 
- Split the data into variables named exactly train, test, and validation. Set the spark randomSplit seed argument to 2019.

In [5]:
medical_df_withna = spark.read.csv("insurance.csv", inferSchema = True, header = True)
medical_df_raw = medical_df_withna.dropna()

sex_pipe = feature.StringIndexer(inputCol='sex', handleInvalid='skip',outputCol="sex_index")
smoker_pipe = feature.StringIndexer(inputCol='smoker', handleInvalid='skip',outputCol="smoker_index")
region_pipe = feature.StringIndexer(inputCol='region', handleInvalid='skip',outputCol = "region_index")
features = Pipeline(stages=[feature.VectorAssembler(inputCols=['age', 'children', 'bmi', 'sex_index', 'smoker_index', 
                                                               'region_index'], outputCol = 'features')])

fe_pipe = Pipeline(stages=[sex_pipe, smoker_pipe, region_pipe, features])
fitted_fe_pipe = fe_pipe.fit(medical_df_raw)
medical_df = fitted_fe_pipe.transform(medical_df_raw)

In [6]:
# train, validation, test = medical_df_raw.randomSplit([0.6, 0.3, 0.1], seed = 2019)

In [7]:
#Print the schema
medical_df.printSchema()
#Print the shape
# print('The shape of the dataframe is:', shape(medical_df))
# print the head
medical_df.show()

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)
 |-- sex_index: double (nullable = false)
 |-- smoker_index: double (nullable = false)
 |-- region_index: double (nullable = false)
 |-- features: vector (nullable = true)

+---+------+------+--------+------+---------+-----------+---------+------------+------------+--------------------+
|age|   sex|   bmi|children|smoker|   region|    charges|sex_index|smoker_index|region_index|            features|
+---+------+------+--------+------+---------+-----------+---------+------------+------------+--------------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|      1.0|         1.0|         2.0|[19.0,0.0,27.9,1....|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|      0.0|         0.0|         0.0|[18.0,1

##### Grading Feedback Cell

The following questions will use deep learning.  The goal is to see if we can improve upon the linear regression score from homework 3, and also compare MSE scores between deep learning and random forest / GBT. You can find the spark documentation for the spark multilayer perceptron classifier can be found [here](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.MultilayerPerceptronClassifier.html).

# Question 2 (10 pts)
Create and train a spark multi layer perceptron model using a grid search in the cell below.  Score your model using MSE.  You are free to use K-Fold Cross validation if you wish.  Your grid search must be entirely encapsulated in the `if enable_grid_search` if statement.  The `enable_grid_search` Boolean is defined in a grading cell above.  You will disable the grid search before you submit by setting enable_grid_search to false.  Setting enable_grid_search to false should not result in a runtime error.  You will not receive full credit if any part of your grid search is outside of the if statement or if runtime errros result from setting the `enable_grid_search` variable to false.

In [8]:
from statistics import median
from pyspark.sql.functions import when
from pyspark.sql.functions import lit
medical_df_pd = medical_df.toPandas()
chagres_median = median(medical_df_pd['charges'])

strat_medical_df = medical_df.withColumn("rate_pool", \
   when((medical_df.charges <= chagres_median), lit(0)) \
     .when((medical_df.charges > chagres_median), lit(1)) \
  )

display(strat_medical_df.show(10))

+---+------+------+--------+------+---------+-----------+---------+------------+------------+--------------------+---------+
|age|   sex|   bmi|children|smoker|   region|    charges|sex_index|smoker_index|region_index|            features|rate_pool|
+---+------+------+--------+------+---------+-----------+---------+------------+------------+--------------------+---------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|      1.0|         1.0|         2.0|[19.0,0.0,27.9,1....|        1|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|      0.0|         0.0|         0.0|[18.0,1.0,33.77,0...|        0|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|      0.0|         0.0|         0.0|[28.0,3.0,33.0,0....|        0|
| 33|  male|22.705|       0|    no|northwest|21984.47061|      0.0|         0.0|         1.0|[33.0,0.0,22.705,...|        1|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|      0.0|         0.0|         1.0|[32.0,0.0,28.88,0...|        0|


None

In [9]:
train, validation, test = strat_medical_df.randomSplit([0.6, 0.3, 0.1], seed = 2019)

In [10]:
mlp = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setFeaturesCol('features').\
    setLabelCol('rate_pool').\
    setLayers([6, 2])

mlp_model = Pipeline(stages=[mlp]).fit(train)

In [11]:
# your code here
if enable_grid_search:
    mlp = classification.MultilayerPerceptronClassifier(seed=2019).\
    setFeaturesCol('features').\
    setLabelCol('rate_pool').\
    setLayers([6, 100, 2])
    
    grid = ParamGridBuilder().\
            addGrid(mlp.maxIter, [10,30,60,80,100]).\
            addGrid(mlp.stepSize, [0.2,0.3,0.4]).\
            build()
    
    all_model = []
    all_pipe = Pipeline(stages=[mlp])
    
    for i in range(len(grid)):
        print("Fitting model: {}".format(i+1))
        model = all_pipe.fit(train, grid[i])
        predictions = model.transform(test)
        
        evaluator = BinaryClassificationEvaluator(labelCol = "rate_pool")
        auc = evaluator.evaluate(predictions)
        all_model.append(auc)
    
    best_model_index = np.argmax(all_model)
    print("The best model index is: ", best_model_index)
    grid[best_model_index]
    print(grid[best_model_index])

##### Grading Feedback Cell

# Question 3 (10 pts)
Create a pipeline named `best_mlp_pipe` that hard codes the tuning parameters from the best model found by the grid search in question 2 above.  Train and test best_mlp_pipe.  Score your model using MSE.  Do not use k-fold cross validation in this question.  Clearly print the resulting **train and test MSE** for `best_mlp_pipe` so it's easy for the graders to see your resulting MSEs.  Save train and test MSE scores in variables named mlp_train_mse and mlp_validation_mse.

In [12]:
maxIter = 60
stepSize = 0.2

best_mlp = classification.MultilayerPerceptronClassifier().\
        setLabelCol('rate_pool').\
        setFeaturesCol('features').\
        setMaxIter(maxIter).\
        setStepSize(stepSize).\
        setLayers([6, 100, 2])

best_mlp_pipe = Pipeline(stages=[best_mlp])
model_q3 = best_mlp_pipe.fit(train)

predictions_q3 = model_q3.transform(test)
evaluator = BinaryClassificationEvaluator(labelCol="rate_pool", rawPredictionCol="prediction", metricName="areaUnderROC")
mlp_train_mse = evaluator.evaluate(predictions_q3)

predictions_q3_val = model_q3.transform(validation)
evaluator_val = BinaryClassificationEvaluator(labelCol="rate_pool", rawPredictionCol="prediction", metricName="areaUnderROC")
mlp_validation_mse = evaluator_val.evaluate(predictions_q3_val)

##### Grading Feedback Cell

In [13]:
# Grading cell do not modify
print("mlp_train_mse =", mlp_train_mse)
print("mlp_validation_mse =", mlp_validation_mse)

mlp_train_mse = 0.872079772079772
mlp_validation_mse = 0.85391686102346


## Question 4 (10 pts)
Create a pandas dataframe named `rf_gbt_mlp_mse_compare` which contains 3 columns: Model, Train MSE, and Test MSE.  Load the Model column with "RF", "GBT", or "MLP". Load the train and validation score columns using model train and validaiton scores (hc_rf_train_mse, hc_rf_validation_mse, hc_gbt_train_mse, hc_gbt_validation_mse, mlp_train_mse, and mlp_validaiton_mse).  

Deep learning might be able to produce better results than decision trees.  I am not sure if that will be the case for this dataset but you will be graded in comparison to other students in the class.

In [14]:
hc_lr_score = 0.93954159140869

In [15]:
data = {'Model': ['LR', 'MLP'],
        'Train MSE': [hc_lr_score, mlp_train_mse]}
rf_gbt_mlp_mse_compare = pd.DataFrame(data)

In [16]:
# Grading Cell Do Not Modify
display(rf_gbt_mlp_mse_compare)

,Model,Train MSE
0,LR,0.939542
1,MLP,0.872080


##### Grading Feedback Cell

# Question 4 (-5 pts if not performed)
Set the `enable_grid_search` Boolean variable to False in the grading cell at the top of this notebook.  Perform a __Runtime -> Disconnect and Delte Runtime__, __Runtime -> Run all__ test to verify there are no runtime errors.  Leave the `enable_grid_search` variable set to False and turn in your assignment.